# Observed Air Quality (PurpleAir)

This notebook retrieves readings from PurpleAir Sensors in Minneapolis and cleans the entries and saves the results as a csv file.

Documentation is available here: https://api.purpleair.com.
You can read this article for help getting started: https://community.purpleair.com/t/making-api-calls-with-the-purpleair-api/180.

From PurpleAir: 

"The data from individual sensors will update no less than every 30 seconds. As a courtesy, we ask that you limit the number of requests to no more than once every 1 to 10 minutes, assuming you are only using the API to obtain data from sensors. If retrieving data from multiple sensors at once, please send a single request rather than individual requests in succession.

The PurpleAir historical API is released as of July 18, 2022. For more information, view this post: https://community.purpleair.com/t/new-version-of-the-purpleair-api-on-july-18th/1251.

Please let us know if you have any questions or concerns, and have a great day!"

A paper on this process: https://doi.org/10.5194/amt-14-4617-2021 (Link for [Download](https://www.researchgate.net/publication/352663348_Development_and_application_of_a_United_States-wide_correction_for_PM25_data_collected_with_the_PurpleAir_sensor) )

Chat on which PM Estimate to use: https://community.purpleair.com/t/pm2-5-algorithms/3972/6

In [1]:
### Import Packages

# File manipulation

import os # For working with Operating System
import requests # Accessing the Web
import datetime as dt # Working with dates/times
import io # Input/Output Bytes objects
import time # For sleep in for loop

# Analysis

import numpy as np
import pandas as pd

## Definitions

In [2]:
spike_threshold = 28 # Micgrograms per meter cubed

In [3]:
# This is my personal API key... Please use responsibly!

api = input('Please enter your Purple Air api key')

Please enter your Purple Air api key 51592903-B445-11ED-B6F4-42010A800007


### Sensor Indices and Start Times

In [4]:
# Sensor Indices (from City of Minneapolis)

sensor_info = pd.read_excel('PA IDs and indexes.xlsx') # Load as DataFrame

sensor_ids = sensor_info['Sensor Index'].dropna().astype(int) # This should be an iterable of the sensor ids as integers

In [5]:
def getSensorsData(query='', api_read_key=''):

    # my_url is assigned the URL we are going to send our request to.
    url = 'https://api.purpleair.com/v1/sensors?' + query
    
    # print('Here is the full url for the API call:\n\n', url)

    # my_headers is assigned the context of our request we want to make. In this case
    # we will pass through our API read key using the variable created above.
    my_headers = {'X-API-Key':api_read_key}

    # This line creates and sends the request and then assigns its response to the
    # variable, r.
    response = requests.get(url, headers=my_headers)

    # We then return the response we received.
    return response

In [6]:
# Get start Times

sensor_string = '%2C'.join(sensor_ids.astype(str))

query = 'fields=date_created&show_only=' + sensor_string

response = getSensorsData(query, api)

response_dict = response.json() # Read response as a json (dictionary)

col_names = response_dict['fields']
data = np.array(response_dict['data'])

sensors_df = pd.DataFrame(data, columns = col_names)

In [7]:
sensors_df.head()

,sensor_index,date_created
0,142718,1642013869
1,142720,1642013875
2,142726,1642013897
3,142724,1642013889
4,142730,1642013916


### Summary Statistics Functions

In [25]:
%run Summary_Functions.py

print('Stat Names:\n\n', summary_stats_names, '\n')
print('Stat Types:\n\n',summary_stats_dtypes, '\n')

print('Function Names:\n\n', summary_stats_functions)

Stat Names:

 ['n_observations', 'pm25_fullDay_mean', 'pm25_fullDay_min', 'pm25_fullDay_minTime', 'pm25_fullDay_max', 'pm25_fullDay_maxTime', 'pm25_fullDay_std', 'pm25_fullDay_minutesAbove12ug', 'pm25_morningRush_mean', 'pm25_morningRush_min', 'pm25_morningRush_minTime', 'pm25_morningRush_max', 'pm25_morningRush_maxTime', 'pm25_morningRush_std', 'pm25_eveningRush_mean', 'pm25_eveningRush_min', 'pm25_eveningRush_minTime', 'pm25_eveningRush_max', 'pm25_eveningRush_maxTime', 'pm25_eveningRush_std', 'pm25_daytimeAmbient_mean', 'pm25_daytimeAmbient_min', 'pm25_daytimeAmbient_minTime', 'pm25_daytimeAmbient_max', 'pm25_daytimeAmbient_maxTime', 'pm25_daytimeAmbient_std', 'pm25_nighttimeAmbient_mean', 'pm25_nighttimeAmbient_min', 'pm25_nighttimeAmbient_minTime', 'pm25_nighttimeAmbient_max', 'pm25_nighttimeAmbient_maxTime', 'pm25_nighttimeAmbient_std'] 

Stat Types:

 [<class 'int'>, <class 'float'>, <class 'float'>, <class 'str'>, <class 'float'>, <class 'str'>, <class 'float'>, <class 'int'>, 

### Functions

In [26]:
# QAQC

def qaqc(df):
    '''This function wil perform some basic QAQC
    '''
    
    clean_df = df.copy()
    
    # Convert timestamp to datetime
    
    clean_df['timestamp'] = pd.to_datetime(clean_df['timestamp'], unit='s')
    
    # Remove obvious error values
    
    clean_df = clean_df[clean_df.pm25 < 1000] 
    
    # Remove NaNs
    
    clean_df = clean_df.dropna()
    
    return clean_df

# Remove and record Spikes

def get_spikes(df, spike_threshold):
    '''This function removes spikes from a dataframe 
    and returns both the new dataframe
    and a separate spike dataframe
    '''
    
    df_w_spikes = df.copy()
    
    condition = (df.pm25 > spike_threshold)
    
    df_w_spikes['is_spike'] = condition
    
    spikes = df_w_spikes[condition].copy()
    
    return df_w_spikes, spikes

# Get Summary Stats

def get_summary_stats(df):
    ''' This is the main function. It will run all of our functions that get summary stats
    and return as a list.
    '''
    
    stats = []
    
    # Run the functions
    
    for f in summary_stats_functions:
        stats += f(df)
    
    return stats

### Set Up Parameters for Query

In [10]:
### Query Strings

# Average string (in minutes) 1440 is 1 day average

avg_string = 'average=10'

# Environmental fields

env_fields = ['pm2.5_cf_1']

env_fields_string = 'fields=' + '%2C%20'.join(env_fields)

# My Header

my_headers = {'X-API-Key': api}

## The Loop

In [11]:
## Iterables

# Dates

# first_date = pd.to_datetime(sensors_df.date_created, unit = 's').min()
# ^ This is just untrue...


first_date = dt.datetime(2022, 6, 15) # June 15th, 2022?

datelist = pd.date_range(start = first_date, 
                         end = dt.datetime.today(),
                        normalize = True)

print('Last Run on ', dt.datetime.today())

Last Run on  2023-04-20 14:16:20.199991


In [18]:
## Initialize Storage

# Daily Summary

cols = ['sensor_index', 'date'] + summary_stats_names

datatypes = [int, str] + summary_stats_dtypes

dtypes = np.dtype(list(zip(cols, datatypes)))

daily_summary_df = pd.DataFrame(np.empty(0, dtype = dtypes))

# Daily Summary (No Spikes)

cols = ['sensor_index', 'date'] + summary_stats_names

datatypes = [int, str] + summary_stats_dtypes

dtypes = np.dtype(list(zip(cols, datatypes)))

daily_summary_no_spikes_df = pd.DataFrame(np.empty(0, dtype = dtypes))

# Spikes

all_spikes_df = pd.DataFrame(np.empty(0, dtype = [('sensor_index', int),
                                                  ('timestamp', pd._libs.tslibs.timestamps.Timestamp),
                                                  ('pm25', float)]
                                 )
                        )

# No Data for sensor

no_data = pd.DataFrame(np.empty(0, dtype = [('sensor_index', int),
                                            ('date', str)
                                           ]))

In [86]:
# Iterate through the days

for i in range(len(datelist)-1): 
    
    # Set up Timestamp for query    
    
    start_timestamp = int(datelist[i].timestamp())
    end_timestamp = int(datelist[i+1].timestamp())
    
    time_string = 'start_timestamp=' + str(start_timestamp) + '&end_timestamp=' + str(end_timestamp)
    
    # Select Sensors that had been created before this date
    
    select_sensors = sensors_df[sensors_df.date_created <= start_timestamp]
    
    sensor_ids = select_sensors.sensor_index
    
    # Iterate through the Sensors
    
    for sensor_id in sensor_ids:
        
        # For skipping to last spot
        
        is_done = (daily_summary_df.sensor_index == int(sensor_id)) & (daily_summary_df.date == str(datelist[i].date()))
        is_no_data = (no_data.sensor_index == int(sensor_id)) & (no_data.date == str(datelist[i].date()))
        
        # If either of these has a true, it has been parsed
        # True is not in both, then we should process
        if (True not in is_done.values) & (True not in is_no_data.values):
            
            ### Actual Loop

            time.sleep(3)

            # Base URL
            base_url = f'https://api.purpleair.com/v1/sensors/{sensor_id}/history/csv?'

            # Put it all together
            query_url = base_url + '&'.join([time_string, avg_string, env_fields_string])

            response = requests.get(query_url, headers=my_headers)

            if response.status_code == 200:

                # Read response as CSV data
                csv_data = response.content.decode('utf-8')

                if csv_data.count('\n') == 1: # There is only one line (empty data)
                    # print(f"No data for sensor {sensor_id} on {datelist[i]}")

                    no_data.loc[len(no_data.index)] = [sensor_id, datelist[i]]

                else:
                    # Parse CSV data into pandas DataFrame
                    df_individual_sensor = pd.read_csv(io.StringIO(csv_data),
                                                       header=0
                                                      )[['time_stamp', 'pm2.5_cf_1']]

                    df_individual_sensor.columns = ['timestamp', 'pm25']

                    # Perform QAQC

                    clean = qaqc(df_individual_sensor)

                    # Remove Spikes & Concatenate to main storage of spikes

                    clean_w_spikes, spikes = get_spikes(clean, spike_threshold)

                    spikes['sensor_index'] = int(sensor_id)

                    all_spikes_df = pd.concat([all_spikes_df, 
                                               spikes[['sensor_index',
                                                        'timestamp',
                                                        'pm25']]
                                              ],
                                               ignore_index=True)

                    # Get Stats (With Spikes)

                    sum_stats = get_summary_stats(clean_w_spikes)

                    # Add to the daily summary dataframe

                    row = [int(sensor_id), str(datelist[i].date())] + sum_stats

                    daily_summary_df.loc[len(daily_summary_df.index)] = row

                    # Get Stats (Without Spikes)

                    no_spikes = clean_w_spikes[clean_w_spikes.is_spike == False]

                    sum_stats = get_summary_stats(no_spikes)

                    # Add to the daily summary dataframe

                    row = [int(sensor_id), str(datelist[i].date())] + sum_stats

                    daily_summary_no_spikes_df.loc[len(daily_summary_no_spikes_df.index)] = row

            else:
                print(f"Error fetching data for sensor {sensor_id}: {response.status_code} on {datelist[i].date()}")
            
    # Save it!?! After a day is processed, yes!

    daily_summary_df.to_csv('daily_summaries.csv', index = False)

    daily_summary_no_spikes_df.to_csv('daily_summaries_no_spikes.csv', index = False)
    
    # For testing a week
    
    # if i == 7:
    #     break

Error fetching data for sensor 145504: 500 on 2022-07-03 00:00:00
Error fetching data for sensor 142744: 500 on 2022-07-16 00:00:00
Error fetching data for sensor 142736: 500 on 2022-08-01 00:00:00
Error fetching data for sensor 143222: 500 on 2022-08-08 00:00:00
Error fetching data for sensor 143666: 500 on 2022-08-10 00:00:00
Error fetching data for sensor 145250: 500 on 2022-08-12 00:00:00
Error fetching data for sensor 142718: 500 on 2022-08-15 00:00:00
Error fetching data for sensor 142732: 500 on 2022-08-17 00:00:00
Error fetching data for sensor 142756: 500 on 2022-08-21 00:00:00
Error fetching data for sensor 157747: 500 on 2022-09-05 00:00:00
Error fetching data for sensor 156605: 500 on 2022-09-07 00:00:00
Error fetching data for sensor 157871: 500 on 2022-09-09 00:00:00
Error fetching data for sensor 142720: 500 on 2022-09-11 00:00:00
Error fetching data for sensor 143226: 500 on 2022-09-15 00:00:00
Error fetching data for sensor 143248: 500 on 2022-09-16 00:00:00
Error fetc

In [87]:
daily_summary_df.tail(10)

# Drop Duplicates
# daily_summary_df = daily_summary_df.drop_duplicates(ignore_index = True).copy()

,sensor_index,date,n_observations,pm25_fullDay_mean,pm25_fullDay_min,pm25_fullDay_minTime,pm25_fullDay_max,pm25_fullDay_maxTime,pm25_fullDay_std,pm25_fullDay_minutesAbove12ug,...,pm25_daytimeAmbient_minTime,pm25_daytimeAmbient_max,pm25_daytimeAmbient_maxTime,pm25_daytimeAmbient_std,pm25_nighttimeAmbient_mean,pm25_nighttimeAmbient_min,pm25_nighttimeAmbient_minTime,pm25_nighttimeAmbient_max,pm25_nighttimeAmbient_maxTime,pm25_nighttimeAmbient_std
11376,157787,2023-04-19,144,5.433618,1.618,06:10:00,11.513,22:10:00,3.002463,0,...,14:20:00,8.441,12:40:00,1.618957,4.719421,3.009,00:10:00,10.839,02:20:00,1.777595
11377,157785,2023-04-19,125,6.45412,1.817,06:30:00,17.559,19:50:00,4.006231,190,...,14:50:00,6.286,13:00:00,0.771088,4.854156,3.504,00:10:00,6.622,01:40:00,0.95074
11378,157837,2023-04-19,144,6.413618,1.85,07:10:00,28.834,00:10:00,4.141073,100,...,14:10:00,6.553,12:30:00,0.877099,8.373079,3.534,03:00:00,28.834,00:10:00,6.837957
11379,157845,2023-04-19,144,8.175861,1.84,07:10:00,59.097,20:10:00,7.060631,350,...,14:50:00,18.413,13:30:00,4.22002,5.750947,2.974,00:00:00,18.033,03:00:00,3.547877
11380,157861,2023-04-19,144,5.888417,1.562,07:10:00,43.404,01:50:00,4.525206,10,...,14:50:00,5.851,13:20:00,0.737749,6.888,2.854,00:10:00,43.404,01:50:00,9.102148
11381,157871,2023-04-19,144,5.436122,1.427,08:00:00,12.1485,19:10:00,3.414521,20,...,14:40:00,5.952,13:00:00,0.774129,3.711553,2.87,00:20:00,5.312,01:10:00,0.712465
11382,157877,2023-04-19,144,5.771819,1.767,07:40:00,12.35,21:50:00,3.410067,10,...,14:50:00,6.378,13:00:00,0.871093,4.327,3.085,02:20:00,7.177,00:20:00,1.161861
11383,157935,2023-04-19,144,4.734097,1.27,07:30:00,10.251,22:00:00,2.87619,0,...,14:50:00,5.497,13:10:00,0.843824,3.041947,2.316,00:30:00,4.216,01:30:00,0.596414
11384,166459,2023-04-19,144,7.49959,2.743,08:50:00,14.324,19:40:00,3.638057,310,...,14:50:00,8.345,12:30:00,1.137382,6.564053,4.056,03:00:00,9.708,00:20:00,1.636587
11385,168327,2023-04-19,144,5.574896,2.036,07:00:00,11.369,20:40:00,3.041771,0,...,14:20:00,6.725,12:40:00,0.815771,4.279632,2.728,00:10:00,7.985,01:30:00,1.250353


In [88]:
daily_summary_no_spikes_df.tail(10)

# To Remove Duplicates
# daily_summary_no_spikes_df = daily_summary_no_spikes_df.drop_duplicates(ignore_index = True).copy()

,sensor_index,date,n_observations,pm25_fullDay_mean,pm25_fullDay_min,pm25_fullDay_minTime,pm25_fullDay_max,pm25_fullDay_maxTime,pm25_fullDay_std,pm25_fullDay_minutesAbove12ug,...,pm25_daytimeAmbient_minTime,pm25_daytimeAmbient_max,pm25_daytimeAmbient_maxTime,pm25_daytimeAmbient_std,pm25_nighttimeAmbient_mean,pm25_nighttimeAmbient_min,pm25_nighttimeAmbient_minTime,pm25_nighttimeAmbient_max,pm25_nighttimeAmbient_maxTime,pm25_nighttimeAmbient_std
11376,157787,2023-04-19,144,5.433618,1.618,06:10:00,11.513,22:10:00,3.002463,0,...,14:20:00,8.441,12:40:00,1.618957,4.719421,3.009,00:10:00,10.839,02:20:00,1.777595
11377,157785,2023-04-19,125,6.45412,1.817,06:30:00,17.559,19:50:00,4.006231,190,...,14:50:00,6.286,13:00:00,0.771088,4.854156,3.504,00:10:00,6.622,01:40:00,0.95074
11378,157837,2023-04-19,143,6.256832,1.85,07:10:00,21.377,00:20:00,3.701964,90,...,14:10:00,6.553,12:30:00,0.877099,7.236361,3.534,03:00:00,21.377,00:20:00,4.849077
11379,157845,2023-04-19,142,7.666049,1.84,07:10:00,24.418,21:10:00,5.351468,330,...,14:50:00,18.413,13:30:00,4.22002,5.750947,2.974,00:00:00,18.033,03:00:00,3.547877
11380,157861,2023-04-19,143,5.62607,1.562,07:10:00,11.787,21:40:00,3.262053,0,...,14:50:00,5.851,13:20:00,0.737749,4.859333,2.854,00:10:00,11.382,02:00:00,2.220053
11381,157871,2023-04-19,144,5.436122,1.427,08:00:00,12.1485,19:10:00,3.414521,20,...,14:40:00,5.952,13:00:00,0.774129,3.711553,2.87,00:20:00,5.312,01:10:00,0.712465
11382,157877,2023-04-19,144,5.771819,1.767,07:40:00,12.35,21:50:00,3.410067,10,...,14:50:00,6.378,13:00:00,0.871093,4.327,3.085,02:20:00,7.177,00:20:00,1.161861
11383,157935,2023-04-19,144,4.734097,1.27,07:30:00,10.251,22:00:00,2.87619,0,...,14:50:00,5.497,13:10:00,0.843824,3.041947,2.316,00:30:00,4.216,01:30:00,0.596414
11384,166459,2023-04-19,144,7.49959,2.743,08:50:00,14.324,19:40:00,3.638057,310,...,14:50:00,8.345,12:30:00,1.137382,6.564053,4.056,03:00:00,9.708,00:20:00,1.636587
11385,168327,2023-04-19,144,5.574896,2.036,07:00:00,11.369,20:40:00,3.041771,0,...,14:20:00,6.725,12:40:00,0.815771,4.279632,2.728,00:10:00,7.985,01:30:00,1.250353


In [91]:
all_spikes_df

# To Remove Duplicates
# all_spikes_df = all_spikes_df.drop_duplicates(ignore_index = True).copy()

,sensor_index,timestamp,pm25
0,143226,2022-06-17 22:10:00,58.910
1,143226,2022-06-17 22:20:00,35.495
2,143226,2022-06-17 22:00:00,319.660
3,142720,2022-06-18 03:20:00,33.595
4,142720,2022-06-18 03:30:00,36.026
...,...,...,...
174239,145242,2023-04-19 23:40:00,34.399
174240,157837,2023-04-19 00:10:00,28.834
174241,157845,2023-04-19 19:40:00,29.648
174242,157845,2023-04-19 20:10:00,59.097


In [96]:
no_data

# To Remove Duplicates
# no_data = no_data.drop_duplicates(ignore_index = True).copy()

,sensor_index,date
0,142718,2022-06-15
1,142720,2022-06-15
2,142726,2022-06-15
3,142730,2022-06-15
4,142728,2022-06-15
...,...,...
7257,145234,2023-04-19
7258,145262,2023-04-19
7259,145502,2023-04-19
7260,145504,2023-04-19


In [97]:
daily_summary_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11386 entries, 0 to 11385
Data columns (total 34 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   sensor_index                   11386 non-null  int64 
 1   date                           11386 non-null  object
 2   n_observations                 11386 non-null  int64 
 3   pm25_fullDay_mean              11280 non-null  object
 4   pm25_fullDay_min               11280 non-null  object
 5   pm25_fullDay_minTime           11280 non-null  object
 6   pm25_fullDay_max               11280 non-null  object
 7   pm25_fullDay_maxTime           11280 non-null  object
 8   pm25_fullDay_std               11264 non-null  object
 9   pm25_fullDay_minutesAbove12ug  11386 non-null  int64 
 10  pm25_morningRush_mean          10985 non-null  object
 11  pm25_morningRush_min           10985 non-null  object
 12  pm25_morningRush_minTime       10985 non-null  object
 13  p

In [98]:
# # Save a test dataframe

# clean_w_spikes.to_csv('example_df.csv', index = False)

# Save it!?!

daily_summary_df.to_csv('daily_summaries.csv', index = False)

daily_summary_no_spikes_df.to_csv('daily_summaries_no_spikes.csv', index = False)

all_spikes_df.to_csv('all_spikes.csv', index = False)

no_data.to_csv('no_data_sensors.csv', index = False)